In [7]:
import pandas as pd
import os

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import SimilarityEvaluator, F1ScoreEvaluator, BleuScoreEvaluator, GleuScoreEvaluator, MeteorScoreEvaluator

In [2]:
from dotenv import load_dotenv
load_dotenv(".env")
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment":os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

In [3]:
import evalprompt.generateApiSpec as generateApiSpec
import os
current_folder = globals()['_dh'][0]
# Generate output from the test dataset, the jsonl file under this output will be used as the evaluation dataset for the prompt
query='Please be professional, and use below infomation to generate an OpenAPI specification documentation with YAML format:'
api_folder = os.path.join(current_folder, "./apisrc")
apispec_dir = os.path.join(current_folder, "./apispec")
output_dir = os.path.join(current_folder, "./output")
generateApiSpec.generateOutputData(query, api_folder, apispec_dir, output_dir)

In [ ]:
import random
randomNum = random.randint(1111, 9999)
similarity_evaluator = SimilarityEvaluator(model_config)
data_path = "./output/output.jsonl"
results = evaluate(
    evaluation_name="Eval-Run-" + str(randomNum),
    data=data_path,
    evaluators={
        "relevance": similarity_evaluator,
    },
    evaluator_config={
        "relevance": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        }
    }
)

In [ ]:
glue_eval = GleuScoreEvaluator()
blue_eval = BleuScoreEvaluator()
f1_eval = F1ScoreEvaluator()
metreo_eval = MeteorScoreEvaluator()
data_path = "./output/output.jsonl"
calc_results = evaluate(
    data=data_path,
    evaluators = {
        "glue": glue_eval,
        'blue': blue_eval,
        'f1': f1_eval,
        'metreo': metreo_eval
    },
    evaluator_config={
        "glue": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        },
        "blue": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        },
        "f1": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        },
        "metreo": {
            "ground_truth":"${data.ground_truth}",
            "response":"${data.response}",
            "query":"${data.query}",
        }
    }
)

In [28]:
pprint(results)

In [15]:
pd.DataFrame(results["rows"])

,inputs.response,inputs.query,inputs.context,inputs.ground_truth,outputs.relevance.similarity,outputs.relevance.gpt_similarity
0,openapi: 3.0.1\ninfo:\n title: Weather Foreca...,"Please be professional, and use below infomati...","[ApiController]\n[Route(""[controller]"")]\npubl...",openapi: 3.0.0\ninfo:\n title: Weather Foreca...,3,3
1,openapi: 3.0.0\ninfo:\n title: Pet Management...,"Please be professional, and use below infomati...",const express = require('express');\nconst app...,openapi: 3.0.0\ninfo:\n title: Pets API\n ve...,3,3
2,openapi: 3.0.0\ninfo:\n title: Flask RESTful ...,"Please be professional, and use below infomati...","from flask import Flask, jsonify, request \nfr...",openapi: 3.0.0\ninfo:\n title: API Specificat...,3,3


In [ ]:
pprint(calc_results)

In [13]:
pd.DataFrame(calc_results["rows"])

,inputs.response,inputs.query,inputs.context,inputs.ground_truth,outputs.glue.gleu_score,outputs.blue.bleu_score,outputs.f1.f1_score,outputs.metreo.meteor_score
0,openapi: 3.0.1\ninfo:\n title: Weather Foreca...,"Please be professional, and use below infomati...","[ApiController]\n[Route(""[controller]"")]\npubl...",openapi: 3.0.0\ninfo:\n title: Weather Foreca...,0.383610,0.375684,0.642857,0.787797
1,openapi: 3.0.0\ninfo:\n title: Pet Management...,"Please be professional, and use below infomati...",const express = require('express');\nconst app...,openapi: 3.0.0\ninfo:\n title: Pets API\n ve...,0.465747,0.458020,0.722467,0.607469
2,openapi: 3.0.0\ninfo:\n title: Flask RESTful ...,"Please be professional, and use below infomati...","from flask import Flask, jsonify, request \nfr...",openapi: 3.0.0\ninfo:\n title: API Specificat...,0.441292,0.429895,0.685714,0.518747
